In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tfd = tf.contrib.distributions
tfb = tfd.bijectors
from tensorflow.keras import layers

In [4]:
tf.enable_eager_execution()

## Settings

In [11]:
DTYPE=tf.float32
NP_DTYPE=np.float32

In [164]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
QMat = np.random.rand(3,5)
# normalize the piecewise PDF such that the integral is 1
QMat = tf.nn.softmax(QMat)
# Insert a 0 to the beginning to match the size to the bin array
QMat = np.insert(QMat,0,0,axis=1)
# Print out values of Q for testing purposes
QMat

array([[0.        , 0.17151878, 0.156443  , 0.14940676, 0.30440095,
        0.21823051],
       [0.        , 0.23376845, 0.23709049, 0.12783502, 0.16175796,
        0.23954808],
       [0.        , 0.27275054, 0.23519319, 0.12784552, 0.15184621,
        0.21236454]])

In [216]:
# network in the coupling layer
def net(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return QMat
    return tf.softmax(layers.stack(x, layers.full_connected, [512, 512, out_size]))

class PiecewiseLinear(tfb.Bijector):
    """
    Piecewise Linear: based on 1808.03856
    """
    def __init__(self, D, d, nbins, layer_id=0, validate_args=False, name="PiecewiseLinear"):
        """
        Args:
            D: number of dimensions
            d: First d units are pass-thru units.
        """
        super(PiecewiseLinear, self).__init__(
            forward_min_event_ndims=1, validate_args=validate_args, name=name
        )
        self.D, self.d = D, d
        self.id = layer_id
        self.nbins = nbins
        self.width = 1.0/self.nbins
        
    def Q(self, xd):
        with tf.variable_scope('Q%d' % self.id, reuse=tf.AUTO_REUSE):
            return net(xd, self.D - self.d, self.nbins)
        
    def pdf(self,x):
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.concat([xd, Q[np.arange(len(Q)),ibins]], axis=1)
        
    def _forward(self, x):
        "Calculate forward coupling layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        yD = (xD*self.nbins-ibins)*Q[np.arange(len(Q)),ibins]+np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins]
        return tf.concat([xd, yD], axis=1)
        
    def _inverse(self, y):
        "Calculate inverse coupling layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(Q,axis=1),tf.transpose(yD),side='right')-1)
        xD = ((yD-np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins])*tf.reciprocal(Q[np.arange(len(Q)),ibins])+np.array(ibins,dtype=np.float32))*self.width
        return tf.concat([yd, xD], axis=1)
    
    def _forward_log_det_jacobian(self, x):
        "Calculate log determinant of Coupling Layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)
    
    def _inverse_log_det_jacobian(self, y):
        "Calculate log determinant of Coupling Layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = np.array(np.floor(yD*self.nbins),dtype=np.int32)
        return -tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)

In [217]:
test = PiecewiseLinear(6,3,5)

In [218]:
forward = test._forward(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))
forward

<tf.Tensor: id=1925, shape=(1, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.32796178, 0.5894042 ,
        0.63578925]])>

In [219]:
test._inverse(forward)

<tf.Tensor: id=1977, shape=(1, 6), dtype=float64, numpy=array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6]])>

In [220]:
test.pdf(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=1982, shape=(1, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.156443  , 0.23709049,
        0.12784552]])>

In [221]:
test._forward_log_det_jacobian(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=1987, shape=(1,), dtype=float64, numpy=array([-0.5229958])>

In [222]:
test._inverse_log_det_jacobian(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=1993, shape=(1,), dtype=float64, numpy=array([0.5229958])>

In [223]:
# network in the coupling layer
def netV(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return VMat

# network in the coupling layer
def netW(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return WMat

class PiecewiseQuadratic(tfb.Bijector):
    """
    Piecewise Quadratic: based on 1808.03856
    """
    def __init__(self, D, d, nbins, layer_id=0, validate_args=False, name="PiecewiseLinear"):
        """
        Args:
            D: number of dimensions
            d: First d units are pass-thru units.
        """
        super(PiecewiseQuadratic, self).__init__(
            forward_min_event_ndims=1, validate_args=validate_args, name=name
        )
        self.D, self.d = D, d
        self.id = layer_id
        self.nbins = nbins
        
    def W(self, xd):
        with tf.variable_scope('W%d' % self.id, reuse=tf.AUTO_REUSE):
            return netW(xd, self.D - self.d, self.nbins)
    
            
    def V(self, xd):
        with tf.variable_scope('V%d' % self.id, reuse=tf.AUTO_REUSE):
            return netV(xd, self.D - self.d, self.nbins)
           
    def pdf(self,x):
        xd, xD = x[:, :self.d], x[:, self.d:]
        W = self.W(xd)
        V = self.V(xd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(W,axis=1),tf.transpose(xD),side='right')-1)
        ibinsp1 = ibins+1
        alpha = (xd-np.cumcum(W,axis=1)[np.arange(len(W)),ibins])/W[np.arange(len(W)),ibins]
        result = (V[np.arange(len(V)),ibinsp1]-V[np.arange(len(V)),ibins])*alpha+V[np.arange(len(V)),ibins]
        return tf.concat([xd, result], axis=1)
        
    def _forward(self, x):
        "Calculate forward coupling layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        W = self.W(xd)
        V = self.V(xd)
        WSum = np.cumsum(W,axis=1)
        VSum = np.cumsum(V,axis=1)
        ibins = tf.transpose(tf.searchsorted(WSum,tf.transpose(xD),side='right')-1)
        ibinsp1 = ibins+1
        alpha = (xd-WSum[np.arange(len(W)),ibins])/W[np.arange(len(W)),ibins]
        yD = alpha**2/2.0*(V[np.arange(len(V)),ibinsp1]-V[np.arange(len(V)),ibins]) \
           + alpha*V[np.arange(len(V)),ibinsp1] \
           + (VSum[np.arange(len(V)),ibinsp1]+V[np.arange(len(V)),ibins])*WSum[np.arange(len(W)),ibins]/2.0
        return tf.concat([xd, yD], axis=1)
        
    def _inverse(self, y):
        "Calculate inverse coupling layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(Q,axis=1),tf.transpose(yD),side='right')-1)
        xD = ((yD-np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins])*tf.reciprocal(Q[np.arange(len(Q)),ibins])+np.array(ibins,dtype=np.float32))*self.width
        return tf.concat([yd, xD], axis=1)
    
    def _forward_log_det_jacobian(self, x):
        "Calculate log determinant of Coupling Layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)
    
    def _inverse_log_det_jacobian(self, y):
        "Calculate log determinant of Coupling Layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = np.array(np.floor(yD*self.nbins),dtype=np.int32)
        return -tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)

In [224]:
test = PiecewiseQuadratic(6,3,5)

In [225]:
test._forward(np.array([0.1,0.2,0.3,0.4,0.5,0.6]).reshape(1,6))

NameError: name 'WMat' is not defined